In [ ]:
!pip install -U openpyxl selenium beautifulsoup4 lxml requests tika

In [ ]:
'''
注意事項:
下載對應的 ChromeDriver (web driver) 到程式檔案同一個目錄下後解壓縮，下載前記得對應版本編號。
連結: https://chromedriver.chromium.org/downloads

參考網頁:
[1] 中研院法學期刊
https://www.iias.sinica.edu.tw/publication_list/9
[2] sqlite3 --- SQLite 数据库 DB-API 2.0 接口模块
https://docs.python.org/zh-tw/3/library/sqlite3.html
[3] Selenium give file name when downloading
https://stackoverflow.com/questions/34548041/selenium-give-file-name-when-downloading
[4] Convert Images to PDF using Python
https://datatofish.com/images-to-pdf-python/
'''


'''
匯入套件
'''
# 網路請求工具
import requests as req

# HTML parser
from bs4 import BeautifulSoup as bs

# 操作 browser 的 API
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 處理下拉式選單的工具
from selenium.webdriver.support.ui import Select

# 強制等待 (執行期間休息一下)
from time import sleep

# pretty-print
from pprint import pprint

# 隨機
from random import randint

# 計時
import time

# 整理 json 使用的工具
import json

# 執行 shell command 的時候用的
import os

# 取得錯誤訊息
import sys, traceback

# 子處理程序，用來取代 os.system 的功能
import subprocess

# 正規表達式
import re

# 編碼
from urllib.parse import quote

# SQLite 資料庫
import sqlite3

# 存取 Excel 的工具
from openpyxl import load_workbook
from openpyxl import Workbook

# 取得系統時間的工具
from datetime import datetime

# 引入 hashlib 模組
import hashlib

# 高階文件操作工具
import shutil

# 檔案剖析工具
from tika import parser

In [ ]:
# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")             #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")        #最大化視窗
my_options.add_argument("--incognito")              #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消通知
my_options.add_argument("--lang=zh-TW")  #設定為正體中文
my_options.add_argument('--disable-gpu')
my_options.add_argument('--disable-software-rasterizer')
my_options.add_argument('--user-agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36"')

# 給 web driver 用的變數
driver = None

# 來源首頁
root_url = 'https://www.iias.sinica.edu.tw'
prefix_url = root_url + '/publication_list'
path_url = prefix_url + '/9'
url = path_url + ''

# 指定 sheet name
folderName = sheetName = 'www_iias_sinica_edu_tw'

# 指定 json 檔名
jsonFileName = f'{folderName}.json'

# 建立儲存檔案用的資料夾
folderPath = f'./{folderName}'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)
    
# 設定 Chrome 下載路徑 (需要絕對路徑)
fullDownloadPath = os.getcwd() + '\\' + folderName

#預設下載路徑
my_options.add_experimental_option("prefs", {
    "download.default_directory": fullDownloadPath,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing_for_trusted_sources_enabled": False,
    "safebrowsing.enabled": False,
    "plugins.always_open_pdf_externally": True
})

# 請求標頭
my_headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36'
}

# 放置爬取的資料
listData = []

# 建立一個 Set 物件，準備 add 所有 tuple，這些 tuple 裡面都有 dict_items 物件
mySet = set()

In [ ]:
'''
函式
'''
# md5 (用來為每一筆資料建立唯一代號)
def md5(string):
    m = hashlib.md5()
    m.update(string.encode("utf-8"))
    return m.hexdigest()

# 初始化設定
def init():
    global listData, mySet
    listData = []
    mySet = set()
    
# 走訪頁面
def visit():
    global listData, mySet
    
    # 取得 html 並轉成 soup 物件
    res = req.get(url = url, headers = my_headers)
    res.encoding = 'utf-8'
    soup = bs(res.text, 'lxml')
    
    # 判斷期刊連結是否存在
    if len( soup.select('div.content a[href*=publication_post]') ) > 0:
        for a in soup.select('div.content a[href*=publication_post]'):
            # 判斷是否重複取得 link
            if a['href'] not in mySet:
                # 加入 set，作為後續判斷
                mySet.add(a['href'])
                
                # 整理資料
                listData.append({
                    "journal_title": "中研院法學期刊",
                    "journal_sub_title": "",
                    "publish_num": a['title'],
                    "link": f"{root_url}/{a['href']}",
                    "sub": []
                })

        # 變數初始化
        mySet = set()
    
# 剖析元素資料
def parse():
    global listData
    
    # 走訪內頁
    for index, myDict in enumerate(listData):
        # 取得 html 與 soup 物件
        res = req.get(url = myDict['link'], headers = my_headers)
        res.encoding = 'utf-8'
        soup = bs(res.text, 'lxml')

        # 變數初始化
        journal_name = publish_date = pdf_link = author = ''
        
        # 取得出版日期
        regexPublishDate = r'\d{4}-\d{1,2}' 
        str_publish_date = soup.select_one('div.content div.publication-info.share ul:not([class])').get_text()
        matchPublishDate = re.search(regexPublishDate, str_publish_date)
        if matchPublishDate != None: 
            listData[index]['publish_date'] = publish_date = matchPublishDate[0]
            
        # 取得 篇名、作者、pdf 連結
        for div in soup.select('div.chapter'):
            # 有 pdf 連結的 div 才進行擷取
            if len(div.select('ul.download li a[href*=pdf]')) > 0:
                # 篇名
                journal_name = div.select_one('h2').get_text()
                
                # 作者
                author = div.select('ul.author li')[0].get_text()
                author = re.sub(r'作者|\s|&nbsp;', '', author)
                
                # pdf 連結
                pdf_link = div.select_one('ul.download li a[href*=pdf]')['href']
                
                # 整理資料
                listData[index]['sub'].append({
                    "id": md5(pdf_link),
                    "journal_name": journal_name,
                    "author": author,
                    "pdf_link": pdf_link
                })
        
        
# 儲存成 json
def saveJson():
    global listData
    with open(f"{folderPath}/{jsonFileName}", "w", encoding="utf-8") as file:
        file.write( json.dumps( listData, ensure_ascii=False, indent=4 ) )

# 建立 .db
def makeDB():
    conn = sqlite3.connect(f"{folderPath}/{folderName}.db")
    cursor = conn.cursor()
    sql = '''
    CREATE TABLE "journals" (
        "sn"    INTEGER,
        "id"    TEXT UNIQUE,
        "journal_title"    TEXT,
        "journal_sub_title"    TEXT,
        "publish_num"    TEXT,
        "publish_date"    TEXT,
        "journal_name"    TEXT,
        "author"    TEXT,
        "link"    TEXT,
        "pdf_link"    TEXT,
        "is_downloaded"    INTEGER,
        "created_at"    TEXT,
        "updated_at"    TEXT,
        PRIMARY KEY("sn" AUTOINCREMENT)
    );
    '''
    cursor.execute(sql)
    conn.commit()
    
    # 關閉 sqlite
    conn.close()
    
    
# 儲存 .db
def saveDB():
    with open(f"{folderPath}/{jsonFileName}", "r", encoding="utf-8") as file:      
        # 取得 json 內容
        strJson = file.read()
        
        # 將 json 轉成 list
        listJson = json.loads(strJson)
        
    # 寫入對話記錄
    conn = sqlite3.connect(f"{folderPath}/{folderName}.db")
    
    # 建立 cursor 物件
    cursor = conn.cursor()

    # 執行 SQL 語法
    try:
        # 查詢特定資料，看看是否已經存在於資料表當中
        sql_query = f'''
        SELECT 1
        FROM journals
        WHERE id = ?
        '''
        
        # 寫入資料
        sql_insert = f'''
        INSERT INTO journals (
            id, journal_title, journal_sub_title, publish_num, publish_date, 
            journal_name, author, link, pdf_link, is_downloaded, 
            created_at, updated_at
        ) VALUES ( 
            ?,?,?,?,?,
            ?,?,?,?,?,
            ?,?
        )
        '''
        
        # 放置準備寫入的資料
        list_insert = []
        
        # 將 json 資料一筆一筆找出來
        for index, myDict in enumerate(listJson):
            for d in listJson[index]['sub']:
                # 如果資料庫沒有這筆資料(透過 id)，則將資料以 tuple 格式放到 list 當中，方便新增 bulk 資料
                if cursor.execute(sql_query, (d["id"],)).fetchone() == None:
                    # 整合所有需要寫入的資料
                    list_insert.append((
                        d['id'],
                        myDict['journal_title'],
                        myDict['journal_sub_title'],
                        myDict['publish_num'],
                        myDict['publish_date'],
                        d['journal_name'],
                        d['author'],
                        myDict['link'],
                        d['pdf_link'],
                        0,
                        datetime.today().strftime("%Y-%m-%d %H-%M-%S"),
                        datetime.today().strftime("%Y-%m-%d %H-%M-%S")
                    ))
        
        # 新增資料到資料庫當中
        cursor.executemany(sql_insert, list_insert)
        
        # 執行 SQL 語法
        conn.commit()
    except sqlite3.Error as err: 
        # 回滾
        conn.rollback()

        # SQLite3 例外處理
        exc_type, exc_value, exc_tb = sys.exc_info()
        strErrorMsg = f'''SQLite error: {' '.join(err.args)}\n\n
        SQLite traceback: {traceback.format_exception(exc_type, exc_value, exc_tb)}
        '''
        print(strErrorMsg)
    finally:
        # 關閉 sqlite
        conn.close()
        
# 下載
def download():
    # 寫入對話記錄
    conn = sqlite3.connect(f"{folderPath}/{folderName}.db")
    
    # 將查詢出來的結果 (tuple)，變成 key-value 型式 (dict)
    conn.row_factory = sqlite3.Row
    
    # 建立 cursor 物件
    cursor = conn.cursor()

    # 執行 SQL 語法
    try:
        # 查詢尚未下載的資料
        sql_query = f'''
        SELECT sn, id, pdf_link
        FROM journals
        WHERE `is_downloaded` = 0
        '''
        
        # 更新資料的欄位(是否下載過)
        sql_update = f'''
        UPDATE `journals` 
        SET 
            `is_downloaded` = 1 ,
            `updated_at` = ?
        WHERE `id` = ?
        '''
            
        # 取得所有未下載的資料
        for myDict in cursor.execute(sql_query).fetchall():
            # 等待
            sleep(randint(1,2))

            # 下載 pdf
            cmd = [
                'curl', 
                '-k', '-L', 
                '-H', 'User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36',
                myDict["pdf_link"], 
                '-o', f'{folderPath}/{myDict["id"]}.pdf'
            ]
            result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
            pprint(result.stdout)
            print("=" * 50)

            # 將 is_downloaded 改成 1，代表已下載過
            cursor.execute(sql_update, (datetime.today().strftime("%Y-%m-%d %H-%M-%S"), myDict["id"],))
                
        conn.commit()
    except sqlite3.Error as err: 
        # 回滾
        conn.rollback()

        # SQLite3 例外處理
        exc_type, exc_value, exc_tb = sys.exc_info()
        strErrorMsg = f'''SQLite error: {' '.join(err.args)}\n\n
        SQLite traceback: {traceback.format_exception(exc_type, exc_value, exc_tb)}
        '''
        print(strErrorMsg)
    finally:
        # 關閉 sqlite
        conn.close()

In [ ]:
init()

In [ ]:
visit()

In [ ]:
parse()

In [ ]:
saveJson()

In [ ]:
makeDB()

In [ ]:
saveDB()

In [ ]:
download()